In [ ]:
# ライブラリ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib as plt

# モデル
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model_selection
from sklearn.model_selection import RepeatedKFold, cross_val_score


### 1. データを読み込む

In [ ]:
df = pd.read_csv("../data/vgsales.csv")
df.head()

### 2. サンプルデータの表示, カラム名

In [ ]:
# サンプルデータの表示
df.sample(10)

In [ ]:
# カラム名の表示
df.columns

### 3. 欠損値の確認

In [ ]:
df.isnull().sum()

### 4. Object typeの確認

In [ ]:
df.info()

In [ ]:
df.dtypes

### 5. 各からむの要約統計量を表示する

In [ ]:
df.describe()

### 6. 各カラム同士の散布図とそれぞれのカラムのヒストグラムを表示する

In [ ]:
sns.pairplot(df)

# ヒストグラムが表示されるが、今回は値がずれているので、表示が小さい

### 7。それぞれのカテゴリのレコード数を確認

In [ ]:
# カテゴリー数
category_columns = df.select_dtypes(include=['object']).columns
df[category_columns].nunique()


### 8. 外れ値の確認

In [ ]:
df[df['Rank']==16600]

In [ ]:
df.sort_values('Global_Sales')

### Publisherごとに売上を集計

In [ ]:

df.groupby('Publisher').sum().sort_values('Global_Sales', ascending=False)[:10].plot.bar(y='Global_Sales')

In [ ]:
df['Year'].value_counts().sort_index()

In [ ]:
df.sort_values('Year', ascending=False)[:10]

In [ ]:
# それぞれのジャンルの数を見る
df['Genre'].value_counts()

In [ ]:
# プラットフォームを見る
df['Platform'].value_counts()

### 9.カラム名の相関

In [ ]:
num_columns = df.select_dtypes(include=['int64', 'float64']).columns
# df[num_columns].corr()
sns.heatmap(df[num_columns].corr(), annot=True, cmap='coolwarm')


In [ ]:
df.sort_values('Global_Sales', ascending=False)[:30]

## Preprocessing

In [ ]:
df = pd.read_csv("../data/vgsales.csv")
df

### 1. 欠損値の数を確認する

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
len(df[df['Publisher'].isna()])

In [ ]:
len(df[df['Year'].isna()])

In [ ]:
# publisherがnanのデータのindexを取得
pub_na_idx = df[df['Publisher'].isna()].index

In [ ]:
# df[[]]とすることでデータフレームのまま代入できる
# fillnaでnanを埋める
df[['Publisher']] = df[['Publisher']].fillna('NaN').iloc[pub_na_idx]

In [ ]:
df.iloc[pub_na_idx]

In [ ]:
# もう一回読み込む
df = pd.read_csv("../data/vgsales.csv")
year_na_idx = df[df['Year'].isna()].index

In [ ]:
# Publisherの欠損値をNaN, Yearの欠損値をYearの中央値で埋める
df.fillna({'Publisher': 'NaN', 'Year': df['Year'].median()}, inplace=True)

In [ ]:
df.iloc[year_na_idx][:5]

In [ ]:
# Unknowも欠損値
# Unknowを他の欠損値と同じように扱うかを考える
pub_nan_df = df[df['Publisher'] == 'NaN']
pub_unknown_df = df[df['Publisher'] == 'Unknown']
pub_missing_df = pd.concat([pub_nan_df, pub_unknown_df])


In [ ]:
pub_missing_df

In [ ]:
# 視覚的にUnknownとNaNの分布の違いを見る
sns.pairplot(pub_missing_df, hue='Publisher')

In [ ]:
# 欠損値を扱うためのライブラリ
from sklearn.impute import SimpleImputer
df = pd.read_csv("../data/vgsales.csv")
# あくまで学習データのみで行う
imputer = SimpleImputer(strategy='median')
df['Year'] = imputer.fit_transform(df[['Year']])

In [ ]:
df.iloc[year_na_idx]

In [ ]:
# publisherも同様にする

imputer = SimpleImputer(strategy="most_frequent")
df['Publisher'] = imputer.fit_transform(df[['Publisher']])[:, 0]


In [ ]:
df.iloc[pub_na_idx]

In [ ]:
### PlatformごとにYearの中央値を計算して埋める
# platform_year_dict = df.groupby('Platform').median()['Year'].to_dict()

In [ ]:
df.apply

In [ ]:
import pandas as pd

# データ読み込み
df = pd.read_csv("../data/vgsales.csv")

# Year列のデータ型を確認し、必要に応じて数値型に変換
if df['Year'].dtype != 'float64' and df['Year'].dtype != 'int64':
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# 欠損値の処理（NaNを除外して中央値を計算）
platform_year_dict = df.groupby('Platform')['Year'].median().to_dict()



In [ ]:
df['Year'] = df.apply(
    lambda row: platform_year_dict[row['Platform']] if np.isnan(row['Year']) and row['Platform'] in platform_year_dict else row['Year'], 
    axis=1
)


In [ ]:
df.iloc[year_na_idx]

### 欠損値対応

### 1. 'Year'の欠損値をKNNで補完する


In [ ]:
# 今回は目的変数はなしとして,knnで欠損値を埋める
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

# 特徴量スケーリング
from sklearn.preprocessing import StandardScaler

# データ分割
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("../data/vgsales.csv")
df.head()

In [ ]:
df.info()

In [ ]:
# Publihserの欠損値を埋める
df[['Publisher']] = df[['Publisher']].fillna('NaN')

In [ ]:
df = pd.read_csv("../data/vgsales.csv")
y_nan_col = 'Year'

# get_dummiesでダミー変数するとnameカテゴリのせいで、カラムが大量に増える
# nameカテゴリがあるが、nameカテゴリは予測に関係ないので、削除する, 過学習を防ぐためにも削除する
df.drop('Name', axis=1, inplace=True)

X = df.drop(y_nan_col, axis=1)
y = df[y_nan_col]


# 数値カラムだけを抽出して、標準化する(get_dummiesでダミー変数をした後に標準化はしない)
num_columns = X.select_dtypes(include=np.number).columns.to_list()

# dummy変数化
X = pd.get_dummies(X, drop_first=True)

# 数値カラムだけ標準化
scaler = StandardScaler()
X[num_columns] = scaler.fit_transform(X[num_columns])

# データ分割
# df['Year']が欠損値のデータのインデックスを取得
test_indexes = df[df['Year'].isna()].index
# df['Year']が欠損値ではないデータのインデックスを取得
train_indexes = df[~df['Year'].isna()].index

X_train, X_test = X.iloc[train_indexes], X.iloc[test_indexes]
y_train, y_test = y.iloc[train_indexes], y.iloc[test_indexes]

# モデルの学習
knn = KNeighborsRegressor(n_neighbors=3).fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
X_test.head(1)

In [ ]:
# このyearに使われたデータを取得する
# 今回はindexが210, 239, 278のデータが使われた
knn.kneighbors(X_test.head(1))

In [ ]:
df.iloc[[210, 239, 278]]

In [ ]:
X_test.head(1)

In [ ]:
# get_dummiesでダミー変数するとnameカテゴリのせいで、カラムが大量に増える
# nameカテゴリがあるが、nameカテゴリは予測に関係ないので、削除する, 過学習を防ぐためにも削除する

# 数値カラムだけを抽出して、標準化する(get_dummiesでダミー変数をした後に標準化はしない)
pd.get_dummies(X, drop_first=True)

### 2. KNNimputerを使って欠損値を埋める

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)

# データフレームで出力するようにする
imputer.set_output(transform='pandas')

# ダミー変数
df = pd.get_dummies(df, drop_first=True)
# 標準化
scaler = StandardScaler()
df[num_columns] = scaler.fit_transform(df[num_columns])

# すべての欠損値を埋める
# fit_transformがあるものは、set_outputでデータフレームで出力するようにできる
df_imputed = imputer.fit_transform(df)





In [ ]:
df_imputed.iloc[test_indexes]

In [ ]:
y_pred

### Charenge

### 欠損値代入の比較

ペンギンデータセット('penguins_size.csv')を使用して、以下の異なる欠損値対応方法による精度を比較します。

#### 比較するケース

1. **欠損値を落としたケース**
   - `.dropna()`
2. **欠損値を新しいカテゴリとしたケース**
   - 数値カラムは中央値で代入
3. **`sklearn.impute.SimpleImputer()`を使用したケース**
   - `.fit_transform()`
4. **欠損値をkNNで予測したケース**
   - カテゴリカルカラムは最頻値で代入
   - `sklearn.impute.KNNImputer(n_neighbors).fit_transform()`

#### モデル

- 任意のモデルを使用可能（回答例ではロジスティック回帰を使用）

#### 評価方法

- 5-fold cross-validationを3回繰り返す
- `Pipeline`や`ColumnTransformer`クラスを使用
- 評価指標はloglossを使用


### Penguins Size Dataset (`penguins_size.csv`)

このデータセットはペンギンのサイズと種別に関する情報を記録しています。

#### データセットの内容

- `species`: ペンギンの種別（Chinstrap, Adelie, Gentoo）
- `culmen_length_mm`: くちばしの長さ (mm)
- `culmen_depth_mm`: くちばしの高さ (mm)
- `flipper_length_mm`: 翼の長さ (mm)
- `body_mass_g`: 体重 (g)
- `island`: 調査した島（Dream, Torgersen, Biscoe） @南極大陸
- `sex`: 性別


In [ ]:
# データ読み込み
df = pd.read_csv("../data/penguins_size.csv")
df

In [ ]:
df.info()

データの確認

In [ ]:
%matplotlib inline
sns.pairplot(df, hue='species')

In [ ]:
# seabornで相関係数を見る
sns.heatmap(df.corr(numeric_only=True), annot=True)


In [ ]:
df.corr(numeric_only=True)

In [ ]:
df.describe()

欠損値をdorpnaで削除するケース

In [ ]:
df['sex'].unique()

In [ ]:
df[df['sex'] == '.']

In [ ]:
# sexからむで.となっているデータをNanとする
df.loc[df[df['sex']=='.'].index, 'sex'] = np.nan

In [ ]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv("../data/penguins_size.csv")
# sexからむで.となっているデータをNanとする
df.loc[df[df['sex']=='.'].index, 'sex'] = np.nan
df.dropna(inplace=True)

target = 'species'
X = df.drop(target, axis=1)
y = df[target]

# ダミー変数の生成(columntransofomerで作成)
# 標準化(columntransofomerで作成)

# columntransofomerでダミー変数と標準化を同時に行う
# 数値カラムとカテゴリカルカラムを分ける
num_columns = X.select_dtypes(include=np.number).columns.to_list()
cat_columns = X.select_dtypes(exclude=np.number).columns.to_list()

# ColumnTransformerの作成

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns),
        ('cat', OneHotEncoder(drop='first'), cat_columns)
    ]
)

# パイプラインの作成
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])

# クロスバイリデーションの設定
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)

# モデルの評価
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_log_loss')

# 平均値を出力
print(f'平均値: {-np.mean(scores)}')



### 答え

In [ ]:
# 結果を格納するでディクショナリー
results = {}

In [ ]:
df = pd.read_csv("../data/penguins_size.csv")
df.loc[df[df['sex']=='.'].index, 'sex'] = np.nan
df.dropna(inplace=True)

target = 'species'
X = df.drop(target, axis=1)
y = df[target]

# ダミー変数
X = pd.get_dummies(X, drop_first=True)

# CV
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_log_loss')
results['drop'] = -np.mean(scores)

欠損値を新カテゴリーとする and 数値カラムは中央値で代入



In [ ]:
df = pd.read_csv("../data/penguins_size.csv")
df.loc[df[df['sex']=='.'].index, 'sex'] = np.nan
df.dropna(inplace=True)


target = 'species'
X = df.drop(target, axis=1)
y = df[target]

cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
# ダミー変数生成クラスを自作(pipelineに入れるため)
from sklearn.base import BaseEstimator, TransformerMixin
class GetDummies(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.columns = None

    def fit(self, X, y=None):
        self.columns = pd.get_dummies(X).columns
        return self

    def transform(self, X):
        X_new = pd.get_dummies(X)
        return X_new.reindex(columns=self.columns, fill_value=0)

# 欠損値を代入処理はcross_val_scoreの中で行う
# pipelineはデータフレーム全体の処理
# 数値カラムだけの処理pipelineでできない→ColumnTransformerで行う
num_columns = X.select_dtypes(include=np.number).columns.to_list()
cat_columns = X.select_dtypes(exclude=np.number).columns.to_list()

ct = ColumnTransformer(
    transformers=[
        ('imputer_num', SimpleImputer(strategy='median', fill_value='Nan'), num_columns),
        ('imputer_cat', SimpleImputer(strategy='constant', fill_value='Nan'), cat_columns)
    ]
)

ct.set_output(transform='pandas')

# pipeline(dummies + scaele + model)
# pipelineはfitとtransformeするものしか入らない
# get_dummies自分でクラスを作る必要がある
pipeline = Pipeline(steps=[
    ('ct', ct),
    ('dummies', GetDummies()),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_log_loss')
results['median'] = -np.mean(scores)
results




In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class GetDummies(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.columns = None

    def fit(self, X, y=None):
        self.columns = pd.get_dummies(X).columns
        return self

    def transform(self, X):
        X_new = pd.get_dummies(X)
        return X_new

In [ ]:
gb = GetDummies()
gb.fit_transform(X)

### 欠損値をKNNで予測したケース(カテゴリカルカラムは最頻値を使用)m

In [ ]:
df = pd.read_csv("../data/penguins_size.csv")
df.loc[df[df['sex']=='.'].index, 'sex'] = np.nan
df.dropna(inplace=True)


target = 'species'
X = df.drop(target, axis=1)
y = df[target]

cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
# ダミー変数生成クラスを自作(pipelineに入れるため)
from sklearn.base import BaseEstimator, TransformerMixin
class GetDummies(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.columns = None

    def fit(self, X, y=None):
        self.columns = pd.get_dummies(X).columns
        return self

    def transform(self, X):
        X_new = pd.get_dummies(X)
        return X_new.reindex(columns=self.columns, fill_value=0)

# 欠損値を代入処理はcross_val_scoreの中で行う
# pipelineはデータフレーム全体の処理
# 数値カラムだけの処理pipelineでできない→ColumnTransformerで行う
num_columns = X.select_dtypes(include=np.number).columns.to_list()
cat_columns = X.select_dtypes(exclude=np.number).columns.to_list()

from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer, make_column_transformer

# KNNimputerの前に標準化する
# 数値カラム用のパイプライン
num_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('imputer_num', KNNImputer())
])

ct = ColumnTransformer(
    transformers=[
        ('imputer_cat', SimpleImputer(strategy='most_frequent'), cat_columns),
        ('num_pipeline', num_pipeline, num_columns)
    ]
)

ct.set_output(transform='pandas')

# pipeline(dummies + scaele + model)
# pipelineはfitとtransformeするものしか入らない
# get_dummies自分でクラスを作る必要がある
pipeline = Pipeline(steps=[
    ('ct', ct),
    ('dummies', GetDummies()),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_log_loss')
results['knn'] = -np.mean(scores)
results


